In [256]:
import pandas as pd
import numpy as np
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
# import datetime as dt
from datetime import datetime, timedelta
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [257]:
raw_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [258]:
main_data = raw_data[0]
main_data['Symbol'] = main_data['Symbol'].str.replace('.', '-')
symbols_list = main_data['Symbol'].unique().tolist()
today = datetime.now().date()
end_date = today - timedelta(days=2)
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)

In [259]:
yf_df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()
yf_df.index.names = ['date', 'ticker']
yf_df.columns = yf_df.columns.str.lower()

[*********************100%%**********************]  503 of 503 completed


In [260]:
yf_df['garman_klass_vol'] = ((np.log(yf_df['high']) - np.log(yf_df['low']))**2)/2-((2*np.log(2)-1)*(np.log(yf_df['adj close'])-np.log(yf_df['open']))**2)
yf_df['rsi'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))
yf_df['bb_low'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
yf_df['bb_mid'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
yf_df['bb_high'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])


## Definiting Fynction to obtain the ATR

In [261]:
def get_atr(df):
    atr = pandas_ta.atr(high=df['high'],
                        low=df['low'],
                        close=df['close'],
                        length=14 # standard lenght for calculating ATR but it can be changed based on the requirements
                        )
    atr_mean = atr.mean()
    atr_std = atr.std()
    normalized_atr = (atr - atr_mean) / atr_std
    return normalized_atr

In [262]:
yf_df['atr'] = yf_df.groupby('ticker', group_keys=False).apply(get_atr)

## Defining Function to calculate MACD

In [263]:
def get_macd(i):
    try:
        macd = pandas_ta.macd(close=i, length= 20).iloc[:,0]
        macd_mean = macd.mean()
        macd_std = macd.std()
        normalized_macd = (macd - macd_mean) / macd_std
        return normalized_macd
    except Exception as e:
        print(f'error : {e}')
        print(f'what is causing the error {i}')

In [264]:
yf_df['macd'] = yf_df.groupby('ticker', group_keys=False)['adj close'].apply(get_macd)

error : 'NoneType' object has no attribute 'iloc'
what is causing the error date        ticker
2023-10-04  VLTO      77.800003
2023-10-05  VLTO      74.449997
2023-10-06  VLTO      77.980003
2023-10-09  VLTO      74.610001
2023-10-10  VLTO      75.000000
2023-10-11  VLTO      74.470001
2023-10-12  VLTO      74.230003
2023-10-13  VLTO      71.849998
2023-10-16  VLTO      74.209999
2023-10-17  VLTO      74.480003
2023-10-18  VLTO      72.449997
2023-10-19  VLTO      70.290001
2023-10-20  VLTO      69.930000
2023-10-23  VLTO      71.900002
2023-10-24  VLTO      72.010002
2023-10-25  VLTO      70.790001
2023-10-26  VLTO      70.000000
2023-10-27  VLTO      67.989998
2023-10-30  VLTO      68.360001
2023-10-31  VLTO      69.000000
2023-11-01  VLTO      67.510002
2023-11-02  VLTO      70.279999
Name: VLTO, dtype: float64


## Getting Dollar Volume

In [265]:
yf_df['dollar_v'] = (yf_df['adj close'] * yf_df['volume']) / 1e6

In [266]:
yf_df

adj close       close        high         low        open  \
date       ticker                                                               
2013-11-05 A        33.627892   36.630901   36.738197   36.373390   36.609444   
           AAL      21.381378   22.680000   22.879999   22.330000   22.750000   
           AAPL     16.378105   18.766071   18.888929   18.678572   18.735001   
           ABBV     31.978905   48.169998   48.759998   48.029999   48.590000   
           ABT      30.803564   37.360001   37.430000   36.669998   36.849998   
...                       ...         ...         ...         ...         ...   
2023-11-02 YUM     124.269997  124.269997  125.730003  122.790001  122.839996   
           ZBH     110.199997  110.199997  110.820000  105.760002  105.860001   
           ZBRA    207.000000  207.000000  207.229996  198.190002  200.460007   
           ZION     33.230000   33.230000   33.349998   31.629999   31.629999   
           ZTS     160.910004  160.910004  161.830002  155.220001  155.309998   

                        volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2013-11-05 A         2206184.0         -0.002738        NaN       NaN   
           AAL       7127300.0         -0.001191        NaN       NaN   
           AAPL    265213200.0         -0.006920        NaN       NaN   
           ABBV      4095400.0         -0.067491        NaN       NaN   
           ABT       9035800.0         -0.012198        NaN       NaN   
...                        ...               ...        ...       ...   
2023-11-02 YUM       1920500.0          0.000228  53.832829  4.765719   
           ZBH       3030300.0          0.000468  49.901056  4.619562   
           ZBRA       758500.0          0.000597  39.800576  5.284366   
           ZION      3026800.0          0.000461  50.660876  3.365809   
           ZTS       4137800.0          0.000385  39.950031  5.028423   

                     bb_mid   bb_high       atr      macd     dollar_v  
date       ticker                                                       
2013-11-05 A            NaN       NaN       NaN       NaN    74.189316  
           AAL          NaN       NaN       NaN       NaN   152.391497  
           AAPL         NaN       NaN       NaN       NaN  4343.689680  
           ABBV         NaN       NaN       NaN       NaN   130.966406  
           ABT          NaN       NaN       NaN       NaN   278.334844  
...                     ...       ...       ...       ...          ...  
2023-11-02 YUM     4.792847  4.819975  0.773156 -0.647181   238.660529  
           ZBH     4.670814  4.722065 -0.014315 -0.994006   333.939051  
           ZBRA    5.350762  5.417157  0.403094 -1.246781   157.009500  
           ZION    3.513678  3.661547  0.428365 -0.996168   100.580563  
           ZTS     5.121820  5.215218  1.214090 -2.983151   665.813413  

[1234469 rows x 14 columns]

## Aggregate to monthly level and filter top 150 most liquid stocks to reduce training time

In [267]:
last_columns = [c for c in yf_df.columns.unique(0) if c not in ['dollar_v', 'volume', 'open', 'high', 'low', 'close']]
df = (pd.concat([yf_df.unstack('ticker')['dollar_v'].resample('M').mean().stack('ticker').to_frame('dollar_v'),
                yf_df.unstack()[last_columns].resample('M').last().stack('ticker')], axis=1)).dropna()

In [268]:
df

dollar_v   adj close  garman_klass_vol        rsi  \
date       ticker                                                         
2013-12-31 A        103.210807   37.641232         -0.002942  65.823440   
           AAL      360.389251   23.804224         -0.000265  54.845351   
           AAPL    5893.826630   17.588905         -0.005293  57.436423   
           ABBV     235.608911   35.059296         -0.065822  61.750387   
           ABT      175.364160   31.603327         -0.014538  57.014641   
...                        ...         ...               ...        ...   
2023-11-30 YUM      288.428534  124.269997          0.000228  53.832829   
           ZBH      245.934884  110.199997          0.000468  49.901056   
           ZBRA     159.967738  207.000000          0.000597  39.800576   
           ZION      85.370800   33.230000          0.000461  50.660876   
           ZTS      883.383543  160.910004          0.000385  39.950031   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2013-12-31 A       3.577314  3.630078  3.682843 -1.046868  0.403628  
           AAL     3.083251  3.202097  3.320942 -0.839183  0.563156  
           AAPL    2.897940  2.920911  2.943882 -0.958603 -0.106636  
           ABBV    3.528295  3.573956  3.619617 -1.104983  0.287023  
           ABT     3.424894  3.463288  3.501683 -1.121240 -0.055212  
...                     ...       ...       ...       ...       ...  
2023-11-30 YUM     4.765719  4.792847  4.819975  0.773156 -0.647181  
           ZBH     4.619562  4.670814  4.722065 -0.014315 -0.994006  
           ZBRA    5.284366  5.350762  5.417157  0.403094 -1.246781  
           ZION    3.365809  3.513678  3.661547  0.428365 -0.996168  
           ZTS     5.028423  5.121820  5.215218  1.214090 -2.983151  

[58881 rows x 9 columns]

### calculating 7-years rolling average of dollar volume

In [269]:
df['dollar_v'] = (df.loc[:, 'dollar_v'].unstack('ticker').rolling(7*12, min_periods=12).mean().stack())
df['dv_rank'] = (df.groupby('date')['dollar_v'].rank(ascending=False))
df = df[df['dv_rank']<=150].drop(['dollar_v', 'dv_rank'], axis=1)

## Getting the monthly returns

In [270]:
# identifying lags and clipping outliers
def get_returns(data):
    threshold = 0.005
    lags = [1,2,3,6,9,12]
    for lag in lags:
        data[f'return_{lag}_m'] = (data['adj close']
                                   .pct_change(lag)
                                   .pipe(lambda x: x.clip(lower=x.quantile(threshold),
                                                          upper=x.quantile(1-threshold)))
                                   .add(1)
                                   .pow(1/lag)
                                   .sub(1) 
                                   )
    return data

df = df.groupby('ticker', group_keys=False).apply(get_returns).dropna()

## Using Fama-French factors to assess how assets are affected by market risk, size, value, profitability, and investment strategies through regression analysis provides valuable insights into their exposure to these influential factors.

In [271]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
               'famafrench',
               start='2010')[0]
factor_data.index = factor_data.index.to_timestamp()
factor_data = factor_data.resample('M').last().div(100)
factor_data.index.name = 'date'

In [272]:
factor_data = factor_data.join(df['return_1_m']).sort_index()

In [273]:
obs = factor_data.groupby('ticker').size()
vs = obs[obs >=10]
factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(vs.index)]

### calculating rolling factor Betas.

In [274]:
betas = (factor_data.groupby('ticker', group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1_m'],
                                     exog=sm.add_constant(x.drop('return_1_m', axis=1)),
                                     window=min(36, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1))
         )


In [275]:
betas.shape[0]

13603

In [276]:
df = (df.join(betas.groupby('ticker').shift()))

In [277]:
# result = df.merge(betas.groupby('ticker').shift(), on='ticker', how='left')


In [278]:
df

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2015-11-30 AAL      39.429932         -0.000966  40.880492  3.672028   
           AAPL     26.960346         -0.003027  53.592878  3.285478   
           ABBV     41.160313         -0.053947  46.995738  3.745051   
           ABT      38.669388         -0.009962  52.539097  3.665571   
           ACN      94.345917         -0.006636  57.567392  4.525864   
...                       ...               ...        ...       ...   
2023-11-30 WBA      21.500000          0.000432  44.354686  3.054455   
           WFC      40.509998          0.000272  51.460203  3.663741   
           WMT     165.520004          0.000102  60.418806  5.054867   
           WYNN     89.750000          0.000435  46.578478  4.456122   
           XOM     109.110001          0.000242  47.969381  4.655703   

                     bb_mid   bb_high       atr      macd  return_1_m  \
date       ticker                                                       
2015-11-30 AAL     3.749832  3.827636  0.024957 -0.721383   -0.105388   
           AAPL    3.328796  3.372114 -0.793558 -0.160527   -0.005804   
           ABBV    3.793320  3.841588 -0.330829  0.059938   -0.023509   
           ABT     3.687430  3.709289 -0.841438  0.209322    0.002678   
           ACN     4.550195  4.574526 -0.915371  0.042134    0.000186   
...                     ...       ...       ...       ...         ...   
2023-11-30 WBA     3.129873  3.205290 -1.262976 -0.438118    0.019924   
           WFC     3.707223  3.750704 -0.339638 -0.430403    0.027688   
           WMT     5.086649  5.118430  0.512983  0.527708    0.012912   
           WYNN    4.509816  4.563510 -1.226231 -0.369325    0.022442   
           XOM     4.698351  4.740999  1.314288 -1.509534    0.030798   

                   return_2_m  return_3_m  return_6_m  return_9_m  \
date       ticker                                                   
2015-11-30 AAL       0.031926    0.019861   -0.003670   -0.015729   
           AAPL      0.037844    0.017564   -0.014506   -0.007686   
           ABBV      0.038557   -0.020295   -0.019629   -0.001641   
           ABT       0.059878   -0.000804   -0.011291   -0.004146   
           ACN       0.050195    0.047565    0.020338    0.022024   
...                       ...         ...         ...         ...   
2023-11-30 WBA      -0.016777   -0.052930   -0.053132   -0.050819   
           WFC       0.000136   -0.003393    0.005699   -0.013161   
           WMT       0.017326    0.005930    0.020727    0.018364   
           WYNN     -0.014497   -0.039802   -0.015285   -0.020195   
           XOM      -0.036691   -0.006275    0.012373    0.001056   

                   return_12_m  Mkt-RF  SMB  HML  RMW  CMA  RF  
date       ticker                                               
2015-11-30 AAL       -0.012729     NaN  NaN  NaN  NaN  NaN NaN  
           AAPL       0.000966     NaN  NaN  NaN  NaN  NaN NaN  
           ABBV      -0.011735     NaN  NaN  NaN  NaN  NaN NaN  
           ABT        0.002516     NaN  NaN  NaN  NaN  NaN NaN  
           ACN        0.020034     NaN  NaN  NaN  NaN  NaN NaN  
...                        ...     ...  ...  ...  ...  ...  ..  
2023-11-30 WBA       -0.044957     NaN  NaN  NaN  NaN  NaN NaN  
           WFC       -0.011417     NaN  NaN  NaN  NaN  NaN NaN  
           WMT        0.008170     NaN  NaN  NaN  NaN  NaN NaN  
           WYNN       0.006284     NaN  NaN  NaN  NaN  NaN NaN  
           XOM        0.000353     NaN  NaN  NaN  NaN  NaN NaN  

[14093 rows x 20 columns]

In [279]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'RF', 'CMA']
df.loc[:, factors] = df.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))
# df[factors] = df.groupby('ticker')[factors].transform(lambda x: x.fillna(x.mean()))
df = df.dropna()
# df.info()

In [280]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 13437 entries, (Timestamp('2015-11-30 00:00:00'), 'AAL') to (Timestamp('2023-11-30 00:00:00'), 'XOM')
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   adj close         13437 non-null  float64
 1   garman_klass_vol  13437 non-null  float64
 2   rsi               13437 non-null  float64
 3   bb_low            13437 non-null  float64
 4   bb_mid            13437 non-null  float64
 5   bb_high           13437 non-null  float64
 6   atr               13437 non-null  float64
 7   macd              13437 non-null  float64
 8   return_1_m        13437 non-null  float64
 9   return_2_m        13437 non-null  float64
 10  return_3_m        13437 non-null  float64
 11  return_6_m        13437 non-null  float64
 12  return_9_m        13437 non-null  float64
 13  return_12_m       13437 non-null  float64
 14  Mkt-RF            13437 non-null  float64
 15  SMB        